# Notebook 6: Time-dependence

<div style="float: right; width: 40%">
    
![](media/AnnulusConvectionModel.png)

</div>





We'll look at a convection problem which couples Stokes Flow with time-dependent advection/diffusion.

The starting point is our previous notebook where we solved for Stokes
flow in a cylindrical annulus geometry. We then add an advection-diffusion 
solver to evolve temperature. The Stokes buoyancy force is proportional to the
temperature anomaly, and the velocity solution is fed back into the 
temperature advection term.


In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio

nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import numpy as np
import sympy
import underworld3 as uw

In [3]:
res = 10
r_o = 1.0
r_i = 0.55

rayleigh_number = 3.0e4

meshball = uw.meshing.Annulus(
    radiusOuter=r_o,
    radiusInner=r_i,
    cellSize=1 / res,
    qdegree=3,
)

# Coordinate directions etc
x, y = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.xR
unit_rvec = meshball.CoordinateSystem.unit_e_0

# Orientation of surface normals
Gamma_N = unit_rvec

In [4]:
# Mesh variables for the unknowns

v_soln = uw.discretisation.MeshVariable("V0", meshball, 2, degree=2, varsymbol=r"{v_0}")
p_soln = uw.discretisation.MeshVariable("p", meshball, 1, degree=1, continuous=True)
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3, continuous=True)

### Create linked solvers

We create the Stokes solver as we did in the previous notebook. 
The buoyancy force is proportional to the temperature anomaly
(`t_soln`). Solvers can either be provided with unknowns as pre-defined
meshVariables, or they will define their own. When solvers are coupled,
explicitly defining unknowns makes everything clearer.

The advection-diffusion solver evolved `t_soln` using the Stokes
velocity `v_soln` in the fluid-transport term. 

### Curved, free-slip boundaries

In the annulus, a free slip boundary corresponds to zero radial 
velocity. However, in this mesh, $v_r$ is not one of the unknowns
($\mathbf{v} = (v_x, v_y)$). We apply a non linear boundary condition that
penalises $v_r$ on the boundary as discussed previously in Example 5. 

In [5]:
stokes = uw.systems.Stokes(
    meshball,
    velocityField=v_soln,
    pressureField=p_soln,
)

stokes.bodyforce = rayleigh_number * t_soln.sym * unit_rvec

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.shear_viscosity_0 = 1
stokes.tolerance = 1.0e-3

stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"

stokes.add_natural_bc(1000000 * Gamma_N.dot(v_soln.sym) * Gamma_N, "Upper")

if r_i != 0.0:
    stokes.add_natural_bc(1000000 * Gamma_N.dot(v_soln.sym) * Gamma_N, "Lower")

In [6]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_fn=v_soln,
    order=2,
    verbose=False,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = 1

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(+1.0, "Lower")
adv_diff.add_dirichlet_bc(-0.0, "Upper")

In [7]:
meshball.view(1)



Mesh # 0: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh



Widget(value='<iframe src="http://localhost:51409/index.html?ui=P_0x31ff24f80_0&reconnect=auto" class="pyvista…

Number of cells: 568

| Variable Name       | component | degree |     type        |
| ---------------------------------------------------------- |
| V0                  |    2      |   2    |     VECTOR      |
| p                   |    1      |   1    |     SCALAR      |
| T                   |    1      |   3    |     SCALAR      |
| psi_star_sl_16_0    |    1      |   3    |     SCALAR      |
| W_16_0              |    1      |   3    |     SCALAR      |
| psi_star_sl_28_0    |    2      |   3    |     VECTOR      |
| psi_star_sl_28_1    |    2      |   3    |     VECTOR      |
| W_28_1              |    2      |   3    |     VECTOR      |
| ---------------------------------------------------------- |


| Boundary Name            | ID    | Min Size | Max Size |
| ------------------------------------------------------ |
| Lower                    | 1     | 70       | 70       |
| Upper                    | 2     | 126      | 126      |
| Centre                   | 10    | 0        |

#### Underworld expressions

Note that 

In [8]:
display(type(stokes.constitutive_model.Parameters.shear_viscosity_0))
display(type(adv_diff.constitutive_model.Parameters.diffusivity))

stokes.constitutive_model.Parameters.shear_viscosity_0.sym

underworld3.function.expressions.UWexpression

underworld3.function.expressions.UWexpression

1

### Initial condition

We need to set an initial condition for the temperature field as the 
coupled system is an initial value problem. Choose whatever works but
remember that the boundary conditions will over-rule values you set on 
the lower and upper boundaries.

In [9]:
# Initial temperature

init_t = 0.1 * sympy.sin(3 * th) * sympy.cos(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)

with meshball.access(t_soln):
    t_soln.data[:, 0] = uw.function.evaluate(init_t, t_soln.coords)

Interpolation for 2706 points / 2706


#### Initial velocity solve

The first solve allows us to determine the magnitude of the velocity field 
and is useful to keep separated to check convergence rates etc. 

For non-linear problems, we usually need an initial guess using a 
reasonably close linear problem. 

`zero_init_guess` is used to reset any information in the vector of 
unknowns (i.e. do not use any initial information if `zero_init_guess==True`).

In [10]:
stokes.solve(zero_init_guess=True)

In [11]:
# Keep the initialisation separate
# so we can run the loop again in a notebook

max_steps = 3
timestep = 0
elapsed_time = 0.0

In [12]:
in_or_not = meshball.points_in_domain(p_soln.coords)
np.count_nonzero(in_or_not == True)

286

In [13]:
p_soln

**Class**: <class 'underworld3.discretisation._MeshVariable'>

**MeshVariable:**

  > symbol:  ${ \hspace{ 0.04pt } {p} }$

  > shape:   $(1, 1)$

  > degree:  $1$

  > continuous:  `True`

  > type:    `SCALAR`

**FE Data:**


  > PETSc field id:  $1$ 

  > PETSc field name:   `p` 

array([[-4817.91532014],
       [-4818.19440709],
       [ 1988.40657856],
       [ 1983.9641256 ],
       [-4928.93105818],
       [-5024.82235852],
       [-5086.78626593],
       [-4985.0777822 ],
       [-4940.54609866],
       [-4812.73162284],
       [-4691.38433646],
       [-4564.93059305],
       [-4556.64893581],
       [-4605.08540098],
       [-4653.18494977],
       [-4808.46861284],
       [-4936.38800584],
       [-5030.35787696],
       [-5051.19397142],
       [-5025.76547163],
       [-4929.53395776],
       [-4671.38774193],
       [-4584.38968303],
       [-4563.66396198],
       [-4546.69012509],
       [-4686.56170008],
       [-4812.47750513],
       [-4943.53033264],
       [-5002.1768576 ],
       [-5075.82532923],
       [-5062.96284233],
       [-4913.31810893],
       [-4803.11981617],
       [-4672.38566018],
       [-4575.14790034],
       [-4540.5382974 ],
       [-4583.59525948],
       [-4673.00656193],
       [ 1899.93894102],
       [ 1833.51667777],


In [14]:
in_or_not = meshball.points_in_domain(adv_diff.DuDt.psi_star[0].coords)

In [15]:
np.count_nonzero(in_or_not == True)

2534

In [16]:
init_t = 0.1 * sympy.sin(3 * th) * sympy.cos(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)

with meshball.access(t_soln):
    t_soln.data[:, 0] = uw.function.evaluate(
        init_t + 0.0001 * t_soln.sym[0], adv_diff.DuDt.psi_star[0].coords
    )

Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2534 points / 2706
shape: (2706,) / (2534,)
Extrapolation for 172 points


In [26]:
adv_diff.solve(timestep=0.01)

Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
TIME: 0.00032210350036621094 / 0.12205886840820312 
0.0002391338348388672 - migration
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2534 points / 2706
shape: (2706,) / (2534,)
Extrapolation for 172 points
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2635 points / 2706
shape: (2706,) / (2635,)
Extrapolation for 71 points
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
TIME: 0.00020194053649902344 / 0.11069416999816895 
0.00021576881408691406 - migration
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2635 points

In [40]:
# Null space ?

for step in range(0, max_steps):

    stokes.solve(zero_init_guess=False)
    delta_t = 5 * adv_diff.estimate_dt()
    adv_diff.solve(timestep=delta_t, _evalf=False)

    timestep += 1
    elapsed_time += delta_t

    if timestep % 5 == 0:
        print(f"Timestep: {timestep}, time {elapsed_time}")

Interpolation for 1 points / 1
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 568 points / 568
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2706 points / 2706
TIME: 0.0002639293670654297 / 0.12475895881652832 
0.0002651214599609375 - migration
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2634 points / 2706
shape: (2706,) / (2634,)
Extrapolation for 72 points
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2634 points / 2706
shape: (2706,) / (2634,)
Extrapolation for 72 points
TIME: 0.0004222393035888672 / 0.27703189849853516 
0.00015687942504882812 - migration
Mesh for evaluations: .meshes/uw_annulus_ro1.0_ri0.55_csize0.1.msh
Interpolation for 2647 points / 2706
shape: (2706,) / (2647,)
Extrapolation for 59 points
Mesh for evaluati

In [41]:
delta_t

0.0017720895600059892

In [42]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)

    skip = 1
    points = np.zeros((meshball._centroids[::skip].shape[0], 3))
    points[:, 0] = meshball._centroids[::skip, 0]
    points[:, 1] = meshball._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        integrator_type=45,
        surface_streamlines=True,
        initial_step_length=0.01,
        max_time=1.0,
        max_steps=500,
    )

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False,
    )

    pl.add_mesh(
        pvstream,
        opacity=1,
        show_scalar_bar=False,
        cmap="Greens",
        render_lines_as_tubes=False,
    )

    pl.export_html("html5/annulus_convection_plot.html")
    # pl.show(cpos="xy", jupyter_backend="trame")

In [43]:
#| fig-cap: "Interactive Image: Convection model output"
from IPython.display import IFrame

IFrame(src="html5/annulus_convection_plot.html", width=500, height=400)

## Exercise - Null space

Based on our previous notebook, can you see how to calculate and (if necessary) remove rigid-body the rotation 
null-space from the solution ? 

The use of a coarse-level singular-value decomposition for the velocity solver should help, in this case, but it's wise to 
check anyway.

```python
    stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
```

## Exercise - Heat flux

Could you calculate the radial heat flux field ? Its surface average value plotted against
time tells you if you have reached a steady state.

Hint:

$$
    Q_\textrm{surf} = \nabla T \cdot \hat{r} + T (\mathbf{v} \cdot \hat{r} )
$$ 

```python
    Q_surf = -meshball.vector.gradient(t_soln.sym).dot(unit_rvec) +\
                    t_soln.sym[0] * v_soln.sym.dot(unit_rvec)
```


